<a href="https://colab.research.google.com/github/asigalov61/Torah-GPT2/blob/master/Torah_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torah GPT2

## All credit for this beautiful colab implementation of char-based GPT2 goes out to Andrej Karpathy on which repo it is based: https://github.com/karpathy/minGPT

## A simple implementation of GPT2 trained on the English Concordant Literal Version of the Torah (Biblical Pentateuch). 

### You are welcome to change the input data/file to your own

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [ ]:
#@title Clone minGPT repo (run only once per session)
!git clone https://github.com/asigalov61/minGPT

In [ ]:
#@title Import all modules and setup logging
%cd /content/minGPT
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [ ]:
#@title Setup functions and procedures
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

        
block_size = 128 # spatial extent of the model for its context

In [ ]:
#@title Specify input text file with training data (do not worry, any text format is fine)
full_path_to_training_text_file = "/content/Torah.txt" #@param {type:"string"}
text = open(full_path_to_training_text_file, 'r').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters

In [ ]:
#@title Create GPT2 model
model_embed_size = 512 #@param {type:"slider", min:0, max:1024, step:64}
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=model_embed_size)
model = GPT(mconf)

In [ ]:
#@title Setup all training parameters
number_of_training_epochs = 1 #@param {type:"slider", min:1, max:5, step:1}
training_batch_size = 256 #@param {type:"slider", min:0, max:512, step:4}
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=number_of_training_epochs, batch_size=training_batch_size, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)


In [ ]:
#@title (OPTION 1) Train the model
trainer.train()

In [ ]:
#@title Save/Re-Save the model from memory
torch.save(model, 'trained-model.pth')

In [ ]:
#@title (OPTION 2) Download a small pre-trained model to try without training. Make sure to change the embeding size to 256 in the previous section for model parameters.
%cd /content/minGPT
!wget https://github.com/asigalov61/Torah-GPT2/raw/master/trained-model.pth

In [ ]:
#@title (OPTION 2) Load existing model/checkpoint
model = torch.load('trained-model.pth')
model.eval()

In [ ]:
#@title Generate text with the trained model
number_of_tokens_to_generate = 2048 #@param {type:"slider", min:0, max:8192, step:128}
creativity_temperature = 0.95 #@param {type:"slider", min:0.05, max:4, step:0.05}
input_promt = "Who is YHVH?" #@param {type:"string"}
# alright, let's sample some character-level Shakespeare
from mingpt.utils import sample
import tqdm

context = input_promt
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, number_of_tokens_to_generate, temperature=creativity_temperature, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

#Congrats! :) You did it :)

## Save the model to Google Drive (Standard GD connect code)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')